# Problem Statement: 
We aim to solve 2D steady-state flow problems in homogeneous porous media using the Physics-Informed Neural Networks (PINN) approach of PDE solutions and the JAX library for pressure  over a rectangular domain.



### **Domain**:
$$ 
\Omega = \{ (x, y) \mid 0 \leq x \leq L_1, \, 0 \leq y \leq L_2 \}
$$



### **Governing Equation**:
$$ 
\frac{\partial^2 P}{\partial x^2} + \frac{\partial^2 P}{\partial y^2} = 0, \quad \text{in } \Omega.
$$ 



### **Boundary Conditions**

**Dirichlet Boundary Conditions** (Prescribed pressure on the left and right boundaries:  
   $$ 
   P(0, y) = P_1, \quad \forall y \in [0, L_2].
   $$

   $$ 
   P(L_1, y) = P_2, \quad \forall y \in [0, L_2].
   $$

**Neumann Boundary Conditions** (No flow on the top and bottom boundaries):  
   $$ 
   \frac{\partial P}{\partial y}(x, 0) = 0, \quad \forall x \in [0, L_1].
   $$
  
   $$ 
   \frac{\partial P}{\partial y}(x, L_2) = 0, \quad \forall x \in [0, L_1].
   $$

We solve this problem using Physics-Informed Neural Networks (PINN) and compare it with the Analytical solution for this configuration.



---


**Developed by**:
Lal Mamud, Postdoc - Subsurface Modeler, Environmental Subsurface Science Group, Energy & Environment Division, Pacific Northwest National Laboratory, Richland, WA, USA.

**Mentors**:
Maruti K. Mudunuru and Satish Karra


## 1. Import required Python libraries 

In [1]:
import os
import sys
import numpy as np
import jax
import jax.numpy as jnp
from jax import random, jit, grad, vmap, jacfwd, jacrev
import optax
import matplotlib.pyplot as plt
from matplotlib import cm
import matplotlib as mpl
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score, mean_squared_error

## 2. Define Domain and Boundary Conditions, and Generate Collocation Points

In [2]:
xmin, xmax = 0.0, 1.0
ymin, ymax = 0.0 , 1.0

# Define the boundary conditions
P1 = 1.0             # at x = 0 for all y_b1, Dirichlet boundary at the left (kPa)  
P2 = 0.9             # at x = 1 for all y, Dirichlet boundary at the right (kPa)
dP_dy1 = 0           # at y = 0 for all x, Neumann boundary at the bottom (kPa)
dP_dy2 = 0           # at y = 1 for all x, Neumann boundary at the top (kPa)

# define the number of points on each boundary and 
N_b = 100          # Number of boundary condition points on each side.
N_r = 5_000        # Number of collocation points within the domain

In [3]:
def generate_BCs_and_colloc(N_b, N_r, ymin, ymax, xmin, xmax, P1, P2, dP_dy1, dP_dy2):
    """
    Generates boundary conditions and collocation points.

    Parameters:
    - N_b: Number of boundary condition points.
    - N_r: Number of collocation points.
    - ymin, ymax: Minimum and maximum values for y.
    - xmin, xmax: Minimum and maximum values for x.
    - left_bc: Boundary condition value at x=0.
    - right_bc: Boundary condition value at x=1.

    Returns:
    - conds: A list of arrays for each boundary condition.
    - colloc: An array of collocation points.
    """
    # Generate random number generator keys
    bk1, bk2, bk3, bk4, r1k, r2k = jax.random.split(jax.random.PRNGKey(0), 6)

    # Left BC: P[0,y] = left_bc
    y_b1 = jax.random.uniform(bk1, minval=ymin, maxval=ymax, shape=(N_b, 1))
    x_b1 = jnp.zeros_like(y_b1)
    bc_1 = jnp.ones_like(y_b1) * P1
    BC_1 = jnp.concatenate([x_b1, y_b1, bc_1], axis=1)

    # Right BC: P[1,y] = right_bc
    y_b2 = jax.random.uniform(bk2, minval=ymin, maxval=ymax, shape=(N_b, 1))
    x_b2 = jnp.ones_like(y_b2)
    bc_2 = jnp.ones_like(y_b1) * P2
    BC_2 = jnp.concatenate([x_b2, y_b2, bc_2], axis=1)

    # Bottom BC: P_y[x,0] = 0
    x_b3 = jax.random.uniform(bk3, minval=xmin, maxval=xmax, shape=(N_b, 1))
    y_b3 = jnp.zeros_like(x_b3)
    bc_3 = jnp.ones_like(x_b3) * dP_dy1
    BC_3 = jnp.concatenate([x_b3, y_b3, bc_3], axis=1)

    # Top BC: P_y[x,1] = 0
    x_b4 = jax.random.uniform(bk4, minval=xmin, maxval=xmax, shape=(N_b, 1))
    y_b4 = jnp.ones_like(x_b4)
    bc_4 = jnp.ones_like(x_b4) * dP_dy1
    BC_4 = jnp.concatenate([x_b4, y_b4, bc_4], axis=1)

    conds = [BC_1, BC_2, BC_3, BC_4]

    # Collocation points
    y_c = jax.random.uniform(r1k, minval=ymin, maxval=ymax, shape=(N_r, 1))
    x_c = jax.random.uniform(r2k, minval=xmin, maxval=xmax, shape=(N_r, 1))
    colloc = jnp.concatenate([x_c, y_c], axis=1)

    return x_b1, y_b1, bc_1, x_b2, y_b2, bc_2, x_b3, y_b3, bc_3, x_b4, y_b4, bc_4, x_c, y_c, conds, colloc


# call the function to create boundary conditions and collocation points
x_b1, y_b1, bc_1, x_b2, y_b2, bc_2, x_b3, y_b3, bc_3, x_b4, y_b4, bc_4, x_c, y_c, conds, colloc \
= generate_BCs_and_colloc(N_b, N_r, ymin, ymax, xmin, xmax, P1, P2, dP_dy1, dP_dy2)

In [4]:
def plot_boundary_conditions(x_b1, y_b1, bc_1, x_b2, y_b2, bc_2, x_b3, y_b3, bc_3, x_b4, y_b4, bc_4, x_c, y_c):
    plt.figure(figsize=(5, 4), dpi=150)
    
    # Calculate vmin and vmax based on all boundary condition values
    all_bc_values = np.concatenate([bc_1, bc_2, bc_3, bc_4])
    vmin, vmax = all_bc_values.min(), all_bc_values.max()
    
    # Plot each set of boundary conditions with dynamic vmin and vmax
    s = 10
    scatter1 = plt.scatter(x_b1, y_b1, c=bc_1, marker='x', vmin=vmin, vmax=vmax, label='P[0,y]', cmap=cm.jet, s=s)
    plt.scatter(x_b2, y_b2, c=bc_2, marker='^', vmin=vmin, vmax=vmax, label='P[1,y]', cmap=cm.jet, s=s)
    plt.scatter(x_b3, y_b3, c=bc_3, marker='*', vmin=vmin, vmax=vmax, label='$\\partial P/\\partial y[x,0]$', cmap=cm.jet, s=s)
    plt.scatter(x_b4, y_b4, c=bc_4, marker='o', vmin=vmin, vmax=vmax, label='$\\partial P/\\partial y[x,1]$', cmap=cm.jet, s=s)
    
    # Plot collocation points
    s = 5
    plt.scatter(x_c, y_c, c='k', marker='.', alpha=0.5, label='Collocation points', s=s)
    
    # Labels and colorbar
    plt.xlabel('$X$')
    plt.ylabel('$Y$')
    cbar = plt.colorbar(scatter1, aspect=30)  # Attach colorbar to the first scatter plot
    cbar.set_label('$Pressure (kPa)$')
    
    # Place the legend outside the plot
    plt.legend(loc='upper center', bbox_to_anchor=(0.5, -0.1), ncol=2, frameon=False)
    plt.tight_layout()

# Call the updated function
#plot_boundary_conditions(x_b1, y_b1, bc_1, x_b2, y_b2, bc_2, x_b3, y_b3, bc_3, x_b4, y_b4, bc_4, x_c, y_c)

## 3. Define PDE Residual

In [5]:
# ∂^2P/∂x^2 + ∂^2P/∂y^2 = 0 
def pde_residual_2d_darcy(x,y,P):
    """
    Compute the PDE residual for the 2D Laplace equation.
    
    Inputs:
    x, y: Coordinates of collocation points (N x 1).
    P: Lambda function representing the neural network solution P(x, y).
    
    Output:
    Residual of the PDE at the collocation points (N x 1).
    """
    P_x = lambda x,y:jax.grad(lambda x,y:jnp.sum(P(x,y)),0)(x,y)
    P_xx = lambda x,y:jax.grad(lambda x,y:jnp.sum(P_x(x,y)),0)(x,y)
    
    P_y=lambda x,y:jax.grad(lambda x,y:jnp.sum(P(x,y)),1)(x,y)
    P_yy=lambda x,y:jax.grad(lambda x,y:jnp.sum(P_y(x,y)),1)(x,y)
    
    return P_xx(x,y) + P_yy(x,y)

## 4. Define Neural Network Architecture

In [6]:
# Define Neural Network Architecture 
def neural_net(params, x, y):
    """
    Forward pass of a neural network to predict P(x, y).
    
    Inputs:
    params: Neural network parameters (list of weights and biases).
    x, y: Input coordinates (N x 1).
    
    Output:
    Predicted P(x, y) values (N x 1).
    """
    X = jnp.concatenate([x, y],axis=1)
    *hidden,last = params
    for layer in hidden :
        X = jax.nn.tanh(X@layer['W']+layer['B'])
    return X@last['W'] + last['B']


# Function to initialize parameters of the neural network (weights and biases)
def init_params(layers):
    """
    Initialize parameters (weights and biases) for a neural network with specified layers.
    
    Args:
        layers: List of integers representing the number of nodes in each layer.
                For example, [2, 20, 20, 1] creates a network with input layer (2 nodes),
                two hidden layers (20 nodes each), and an output layer (1 node).
    
    Returns:
        params: List of dictionaries containing 'W' (weights) and 'B' (biases) for each layer.
    """
    keys = jax.random.split(jax.random.PRNGKey(0), len(layers) - 1)  # Generate random keys for each layer
    params = list()  # Initialize a list to store layer parameters
    
    for key, n_in, n_out in zip(keys, layers[:-1], layers[1:]):  # Loop through layer dimensions
        lb, ub = -(1 / jnp.sqrt(n_in)), (1 / jnp.sqrt(n_in))  # Xavier initialization bounds
        W = lb + (ub - lb) * jax.random.uniform(key, shape=(n_in, n_out))  # Initialize weights
        B = jax.random.uniform(key, shape=(n_out,))  # Initialize biases
        params.append({'W': W, 'B': B})  # Append layer parameters (weights and biases) to the list
    
    return params  # Return the initialized parameters

## 5. Define Loss Function

In [7]:
@jax.jit
def loss_fun(params, colloc, conds):
    """
    Calculate the total loss for the PINN.
    
    Inputs:
        params: Neural network parameters (list of weights and biases).
        colloc: Collocation points for the PDE residual (N x 2).
        conds: Boundary condition data (list of 4 elements, one for each BC).
    
    Output:
        Scalar total loss value.
    """
    x_c, y_c = colloc[:, [0]], colloc[:, [1]]
    
    P_nn = lambda x, y: neural_net(params, x, y)
    
    pde_loss = jnp.mean(pde_residual_2d_darcy(x_c, y_c, P_nn)**2)
    
    # Loss at the left and right Dirichlet BCs 
    dbc_loss = 0
    for cond in conds[0:2]:
        x_b, y_b, u_b = cond[:, [0]], cond[:, [1]], cond[:, [2]]  
        #dbc_loss += MSE(P_nn(x_b, y_b), u_b)  # Dirichlet BC loss
        dbc_loss += jnp.mean(P_nn(x_b, y_b) - u_b)**2  # Dirichlet BC loss
    
    # Loss at the bottom and top Neumann BCs 
    nbc_loss = 0
    P_nn_y = lambda x, y: jax.grad(lambda x, y: jnp.sum(P_nn(x, y)), 1)(x, y)  # Neumann BCs derivative
    for cond in conds[2:4]:
        x_b, y_b, u_b = cond[:, [0]], cond[:, [1]], cond[:, [2]]  
        #nbc_loss += MSE(P_nn_y(x_b, y_b), u_b)  # Neumann BC loss
        nbc_loss += jnp.mean(P_nn_y(x_b, y_b) - u_b)**2  # Neumann BC loss

    # Total loss (summed components)
    loss = pde_loss + dbc_loss + nbc_loss
    
    return loss

## 6. Define Hyperparameters and Training Loop
This cell contains the training loop for the Physics-Informed Neural Network (PINN). It includes functions and code to update the network parameters, compute the loss, track training progress, and identify the best-performing model during the training process.

In [8]:
# Define hyperparameters for the neural network and training
hidden_nodes = 20      # Number of nodes in each hidden layer
hidden_layers = 3      # Number of hidden layers in the neural network
lr = 1e-3              # Learning rate for the optimizer
epochs = 2000          # Number of epochs for training


# Construct the MLP (Multilayer Perceptron) using the specified architecture
params = init_params([2] + [hidden_nodes] * hidden_layers + [1])  
# The network has 2 input nodes, `hidden_layers` layers with `hidden_nodes`, and 1 output node.

# Define the optimizer (Adam optimizer with the specified learning rate)
optimizer = optax.adam(lr)

# Initialize the optimizer state using the network parameters
opt_state = optimizer.init(params)




# Parameters update function
@jax.jit
def update(opt_state,params,colloc,conds):
    """
    Perform a single optimization step:
    - Calculate gradients of the loss with respect to network parameters.
    - Update the parameters using the optimizer.
    
    Inputs:
    opt_state: Current optimizer state.
    params: Current neural network parameters (weights and biases).
    colloc: Collocation points for the PDE residual loss.
    conds: Boundary condition data for Dirichlet and Neumann conditions.
    
    Outputs:
    opt_state: Updated optimizer state.
    params: Updated network parameters.
    """
    # Get the gradient w.r.t to MLP params
    grads=jax.jit(jax.grad(loss_fun,0))(params,colloc,conds)
    
    #Update params
    updates, opt_state = optimizer.update(grads, opt_state)
    params = optax.apply_updates(params, updates)
    return opt_state, params


#%% PINN training loop
print('PINN training started...')

# Initialize tracking variables
best_params = params
best_loss = float('inf')
best_epoch = 0

# Initialize lists to track all losses and epochs
all_losses = []
all_epochs = []

# # Training loop
# for epoch in range(epochs+1):
#     # Update the optimizer state and parameters
#     opt_state, params = update(opt_state, params, colloc, conds)

#     # Calculate the current loss
#     current_loss = loss_fun(params, colloc, conds)
    
#     # Store loss and epoch
#     all_losses.append(current_loss)
#     all_epochs.append(epoch)
    
#     # Update the best parameters if the current loss is the lowest
#     if current_loss < best_loss:
#         best_loss = current_loss
#         best_params = params
#         best_epoch = epoch

#     # Print loss and epoch info
#     if epoch % 100 == 0:
#         print(f'Epoch={epoch}\tloss={current_loss:.3e}')

# print('PINN training done!')
# print(f'Best Epoch = {best_epoch}\tBest Loss = {best_loss:.3e}')

PINN training started...


## 7. Plot pinn loss during training

In [9]:
#function for plotting for PINN losses during training 
def plot_pinn_training(all_losses, all_epochs):
    plt.figure(figsize = (5, 4), dpi = 150)
    plt.semilogy(all_epochs, all_losses, '-r', markersize = 4, linewidth = 1.0)
    plt.xlabel("Epochs")
    plt.ylabel("Loss")
    #plt.legend(prop={"size": fontsize_legend}, loc="best")
    plt.xlim(min(all_epochs), max(all_epochs))
    plt.tight_layout()
    
#plot_pinn_training(all_losses, all_epochs)

## 8. PINN prediction and plotting results

In [10]:
#%% PINN prediction and plotting results
nx = 20
x = np.linspace(xmin, xmax, nx)
dx = x[1] - x[0]
ny = 20
y = np.linspace(ymin, ymax, ny)
dy = y[1] - y[0]
X, Y = np.meshgrid(x, y)

# pinn_P = (neural_net(best_params, X.flatten().reshape(-1,1), Y.flatten().reshape(-1,1))).reshape(nx,ny) 
# print(pinn_P)

In [11]:
def plot_2d_pressure_distribution(X, Y, pressure, title, cbar_lebel):
    plt.figure(figsize = (5, 4), dpi = 150)
    cmap = 'jet'
    contour = plt.contourf(X, Y, pressure, levels=100, cmap=cmap)  
    plt.xlabel('$X$')
    plt.ylabel('$Y$')
    # Colorbar settings
    vmin = np.min(pressure)
    vmax = np.max(pressure)
    #cbar = plt.colorbar(mpl.cm.ScalarMappable(norm=mpl.colors.Normalize(vmin = vmin, vmax = vmax), cmap=cm.jet), aspect=30)
    cbar = plt.colorbar(contour, aspect=30)


    cbar.set_label(f'{cbar_lebel}')
    plt.xlim(np.min(X), np.max(X))
    plt.ylim(np.min(Y), np.max(Y))
    plt.title(title)
    plt.tight_layout()

title = "PINN prediction"
cbar_lebel = "$Pressure (kPa)$"
#plot_2d_pressure_distribution(X, Y, pinn_P, title, cbar_lebel)

## 9. Solution verifications

In [12]:
#%% Analytical solution
L = xmax
analytical_P = P1 + (P2 - P1) * X / L
#print(analytical_P)

In [13]:
title = "Analytical solution"
cbar_lebel = "$Pressure (kPa)$"
#plot_2d_pressure_distribution(X, Y, analytical_P, title, cbar_lebel)

## 10. Error analysis between the analytical solution and PINN prediction

In [14]:
# # Flatten the 2D arrays into 1D
# analytical_Pr = analytical_P.flatten().reshape(-1, 1)  # Flatten to 1D and reshape to 2D (n_samples, n_features)
# pinn_Pr = pinn_P.flatten()                       # Flatten to 1D (n_samples,)

# # Perform regression
# reg = LinearRegression()
# reg.fit(analytical_Pr, pinn_Pr)  # Fit the model

# # Predict using the regression model
# y_pred = reg.predict(analytical_Pr)

# # Calculate R^2 score
# r2 = r2_score(pinn_Pr, y_pred)

# # Calculate RMSE
# mse = mean_squared_error(pinn_Pr, y_pred)  # Mean Squared Error
# rmse = np.sqrt(mse)  # Root Mean Squared Erro

# # Print results
# print(f"R^2 Score: {r2:.4f}")
# print(f"Root Mean Squared Error (RMSE): {rmse:.4f}")

# #%% Plotting Regression
# plt.figure(figsize=(5, 4), dpi = 150)
# plt.scatter(analytical_Pr, pinn_Pr, label="Data Points", color="blue", alpha=0.5, s=40)
# plt.plot(analytical_Pr, y_pred, label="Regression Line", color="red", linewidth=2)

# # Add annotations for RMSE and R^2
# plt.text(
#     0.05,
#     0.95,
#     f"$R^2$ = {r2:.4f}\nRMSE = {rmse:.4f}",
#     transform=plt.gca().transAxes,
#     fontsize=12,
#     verticalalignment="top",
#     bbox=dict(boxstyle="round", facecolor="white", alpha=0.8),
# )

# plt.xlabel("Analytical Solution (Flattened)")
# plt.ylabel("PINN Prediction (Flattened)")
# plt.title("Regression Plot (Flattened 2D)")
# plt.legend(loc="lower right")

In [16]:
# # Plot the percentage error
# error_percentage = 100 * (analytical_P - pinn_P) / (analytical_P)
# title = "Error between analytical solution and PINN prediction"
# cbar_lebel = "$Error (\%)$"
#plot_2d_pressure_distribution(X, Y, error_percentage, title, cbar_lebel)